In [2]:
# Set the new configuration
conf = SparkConf().setAll([('spark.executor.memory', '4g'),\
                           ('spark.driver.memory', '4g'),\
                           ('spark.shuffle.service.enabled', True), \
                           ('spark.dynamicAllocation.enabled', True), \
                           #('spark.executor.instances', 50)
                           ('spark.dynamicAllocation.executorIdleTimeout', 600), \
                           ('spark.driver.maxResultSize', 0), \
                           ('spark.executor.cores', 4),\
                           ('spark.default.parallelism', 90),\
                           ('spark.executor.memoryOverhead', '4g'),\
                           ('spark.driver.memoryOverhead', '4g'),\
                           ('spark.scheduler.mode', 'FAIR'),\
                           ('spark.kryoserializer.buffer.max', '512m'),\
                           ('spark.app.name','FifthModelTest - JupyterHub version')])# Show the current options




#                           ('spark.dynamicAllocation.maxExecutors', 90), \


# Stop the old context
sc.stop()

# And restart the context with the new configuration
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import math
import copy
import random
import time
import sys

from pyspark import SparkConf,SparkContext
from pyspark.sql import Row, SQLContext, SparkSession
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.types import *
from pyspark.ml.linalg import Vectors

In [4]:
from keras.layers import *
from keras.models import Model, load_model
from keras.optimizers import Adam, Nadam, SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [5]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import normalize
from sklearn.metrics import confusion_matrix

In [6]:
import tensorflow as tf

In [7]:
sqlContext.sql("use plasticc")

DataFrame[]

In [8]:
augment_count = 25
batch_size = 1000
batch_size2 = 5000
optimizer = 'nadam'
num_models = 1
use_specz = False
valid_size = 0.1
max_epochs = 1000

limit = 1000000
sequence_len = 256

In [9]:
classes = np.array([6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95, 99], dtype='int32')
class_names = ['class_6','class_15','class_16','class_42','class_52','class_53','class_62','class_64','class_65','class_67','class_88','class_90','class_92','class_95','class_99']
class_weight = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1, 99: 1}

# LSST passbands (nm)  u    g    r    i    z    y      
passbands = np.array([357, 477, 621, 754, 871, 1004], dtype='float32')

In [10]:
def get_wtable(df):
    #x=np.array(raw_vectorsDF.select('target').collect())
    
    all_y = np.array(df.select('target').collect(), dtype = 'int32')

    y_count = np.unique(all_y, return_counts=True)[1]

    wtable = np.ones(len(classes))

    for i in range(0, y_count.shape[0]):
        wtable[i] = y_count[i] / all_y.shape[0]

    return wtable

In [11]:
def get_keras_data(sc,vectors_df):
    idArr=np.array(vectors_df.select('object_id').collect(), dtype='float32')
    r, c = idArr.shape
    r,c=idArr.shape
    idArr.reshape(r,)
    meta_len=10
    
    metaArr=np.array(vectors_df.select('meta').collect(), dtype='float32').reshape(r,meta_len)
    bandArr= np.array(vectors_df.select('band').collect() , dtype='int32').reshape(r,sequence_len)

    histArray=np.zeros((r,sequence_len,8), dtype='float32') 
    # this will work brilliantly as get_keras_data sets three columns to zeros anyway

    #mjdInt=np.array(vectors_df.select('hist.interval').collect(), dtype='float32').reshape(r,sequence_len)
    deltaMjd=np.array(vectors_df.select('hist.deltaMjd').collect(), dtype='float32').reshape(r,sequence_len)
    rval=np.array(vectors_df.select('hist.rval').collect(), dtype='float32').reshape(r,sequence_len)
    fluxTest=np.array(vectors_df.select('hist.flux').collect(), dtype='float32').reshape(r,sequence_len)
    flux_err_test=np.array(vectors_df.select('hist.flux_err').collect(), dtype='float32').reshape(r,sequence_len)
    #detected=np.array(vectors_df.select('hist.interval').collect(), dtype='float32').reshape(r,sequence_len)
    source_wavelength=np.array(vectors_df.select('hist.interval').collect(), dtype='float32').reshape(r,sequence_len)
    #received_wavelength=np.array(vectors_df.select('hist.interval').collect(), dtype='float32').reshape(r,sequence_len)
    
    #as per the baseline program, we remove the abs time, detected and receoved_wavelength data

    #histArray[:,:,0]=mjdInt
    histArray[:,:,1]=fluxTest
    histArray[:,:,2]=flux_err_test
    #histArray[:,:,3]=detected
    histArray[:,:,4]=deltaMjd
    histArray[:,:,5]=rval
    histArray[:,:,6]=source_wavelength
    #histArray[:,:,7]=received_wavelength
    
    # Create the final vector dictionary
    X = {
            'id': idArr,
            'meta': metaArr,
            'band': bandArr,
            'hist': histArray
        }
    # and the encoded target vector
    Y = to_categorical(np.array(vectors_df.select('target').collect(), dtype='int32'), num_classes=len(classes))  
    
    return X, Y

In [17]:
idArr=np.array(raw_vectorsDF.select('object_id').collect(), dtype='float32')

In [20]:
X, Y = get_keras_data(sc,raw_vectorsDF)

In [12]:
def train_model(i, samples_train, samples_valid):
    start_augment=time.time()
    start_augmentCpu=time.clock()
    
    #samples_train += augmentate(samples_train, augment_count, augment_count)
    
    elapsed_augment=time.time() - start_augment
    elapsed_augmentCpu=time.clock() - start_augmentCpu

    patience = 1000000 // len(samples_train) + 5

    start_trainingVectors=time.time()
    start_trainingVectorsCpu=time.clock()

    train_x, train_y = get_keras_data(samples_train)

    elapsed_training_Vectors=time.time() - start_trainingVectors
    elapsed_training_VectorsCpu=time.clock() - start_trainingVectorsCpu

    print(len(samples_train))
    
    del samples_train
    
    start_validationVectors=time.time()
    start_validationVectorsCpu=time.clock()

    valid_x, valid_y = get_keras_data(samples_valid)
    del samples_valid
    
    elapsed_validation_Vectors=time.time() - start_validationVectors
    elapsed_validation_VectorsCpu=time.clock() - start_validationVectorsCpu
    
    return  elapsed_augment,elapsed_augmentCpu,\
            elapsed_training_Vectors,elapsed_training_VectorsCpu,\
            elapsed_validation_Vectors,elapsed_validation_VectorsCpu, \
            train_x, train_y

    
    ## THIS IS AS FAR AS WE NEED TO GO FOR THIS TEST


In [13]:
print('Loading train data from hive...')

start_train=time.time()
start_trainCpu=time.clock()

raw_vectorsSQL="""
select * from training_set_augmented_vectors
"""

raw_vectorsDF=sqlContext.sql(raw_vectorsSQL)

elapsed_train=time.time()-start_train
elapsed_trainCpu=time.clock()-start_trainCpu
#wtable = get_wtable(raw_vectorsDF)

Loading train data from hive...


In [14]:
raw_vectorsDF.printSchema()

root
 |-- object_id: integer (nullable = true)
 |-- meta: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- target: integer (nullable = true)
 |-- specz: double (nullable = true)
 |-- band: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: double (containsNull = true)
 |-- hist: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- interval: array (nullable = true)
 |    |    |    |-- element: array (containsNull = true)
 |    |    |    |    |-- element: double (containsNull = true)
 |    |    |-- deltaMjd: array (nullable = true)
 |    |    |    |-- element: array (containsNull = true)
 |    |    |    |    |-- element: double (containsNull = true)
 |    |    |-- rval: array (nullable = true)
 |    |    |    |-- element: array (containsNull = true)
 |    |    |    |    |-- element: double (containsNull = true)
 |    |    |-- flux: array (nullable = true)
 |    |    |    |-- element:

In [ ]:
print(elapsed_train)
print(elapsed_trainCpu)

In [ ]:
start_samples=time.time()
start_samplesC=time.clock()
    
# No get data 
elapsed_samples=time.time() - start_samples
elapsed_samplesC=time.clock() - start_samplesC

In [ ]:
for i in range(1, num_models+1):

    samples_train, samples_valid = train_test_split(samples, test_size=valid_size, random_state=42*i)
    len(samples_train)
    
    start_train=time.time()
    elapsed_augment,elapsed_augmentCpu,\
            elapsed_training_Vectors,elapsed_training_VectorsCpu,\
            elapsed_validation_Vectors,elapsed_validation_VectorsCpu, \
            train_x, train_y = \
            train_model(i, samples_train, samples_valid)
    elapsed_train=time.time()-start_train
    print(elapsed_train)
    #break

In [ ]:
samples_train, samples_valid = train_test_split(samples, test_size=valid_size, random_state=42*1)

In [ ]:
train_x, train_y = get_keras_data(samples_train)

In [ ]:
X=train_x
Y=train_y

In [ ]:
shape=X['hist'][0].shape
shape

In [ ]:
X['meta'][0].shape

In [ ]:
hist_input = Input(shape=X['hist'][0].shape, name='hist')
meta_input = Input(shape=X['meta'][0].shape, name='meta')
band_input = Input(shape=X['band'][0].shape, name='band')

In [ ]:
band_emb = Embedding(8, 8)(band_input)

In [ ]:
hist_input

In [ ]:
band_emb

In [ ]:
hist = concatenate([hist_input, band_emb])
hist = TimeDistributed(Dense(40, activation='relu'))(hist)


In [ ]:
hist

In [ ]:
raw_vectorsDF.select("meta").toPandas().to_numpy()

# Stuff below is final vector analysis

In [ ]:
raw_vectorsDF.count()

In [ ]:
list_persons

#### Converting dataframes to dictionaries
https://stackoverflow.com/questions/41206255/convert-pyspark-sql-dataframe-dataframe-type-dataframe-to-dictionary

In [15]:
list_objects = map(lambda row: row.asDict(), raw_vectorsDF.collect())

Py4JJavaError: An error occurred while calling o159.collectToPython.
: java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.sql.execution.SparkPlan$$anon$1.next(SparkPlan.scala:280)
	at org.apache.spark.sql.execution.SparkPlan$$anon$1.next(SparkPlan.scala:276)
	at scala.collection.Iterator$class.foreach(Iterator.scala:893)
	at org.apache.spark.sql.execution.SparkPlan$$anon$1.foreach(SparkPlan.scala:276)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeCollect$1.apply(SparkPlan.scala:298)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeCollect$1.apply(SparkPlan.scala:297)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:186)
	at org.apache.spark.sql.execution.SparkPlan.executeCollect(SparkPlan.scala:297)
	at org.apache.spark.sql.Dataset$$anonfun$collectToPython$1.apply(Dataset.scala:3200)
	at org.apache.spark.sql.Dataset$$anonfun$collectToPython$1.a

In [ ]:
object_vectors = {object['object_id']: object for object in list_objects}

In [ ]:
samples=[]

In [ ]:
use_specz=False

for key in object_vectors.keys():
    print(key)
    i=object_vectors.get(key)
    
    id=i.get('object_id')

    sample = {}
    sample['id'] = int(id)
    
    # 'object_id', 'target', 'meta', 'specz', 'band', 'hist'
    
    sample['target'] = int(i.get('target'))
    
    meta=np.array(i.get('meta'), dtype='float32')
  
    sample['meta'] = np.zeros(10, dtype = 'float32')

    sample['meta'][4] = meta[0]
    sample['meta'][5] = meta[2]
    sample['meta'][6] = meta[3]
    sample['meta'][7] = meta[4]
    sample['meta'][8] = float(meta[2]) > 0

    sample['specz'] = float(meta[1])    
    
    if use_specz:
        sample['meta'][5] = float(meta['hostgal_specz'])
        sample['meta'][6] = 0.0

    z = float(sample['meta'][5])

    j=i.get('hist')
    mjd=np.array(j[0][0], dtype='float32')
    r,c=mjd.shape
    mjd.reshape(c,)
    band=np.array(  j[0][1], dtype='int32').reshape(c,) # passband
    flux=np.array( j[0][2], dtype='float32').reshape(c,) # flux
    flux_err=np.array( j[0][3], dtype='float32').reshape(c,) # flux_err
    detected=np.array( j[0][4], dtype='int32').reshape(c,) # Detected

    mjd -= mjd[0]
    mjd /= 100 # Earth time shift in day*100
    mjd /= (z + 1) # Object time shift in day*100


    received_wavelength = passbands[band] # Earth wavelength in nm
    received_freq = 300000 / received_wavelength # Earth frequency in THz
    source_wavelength = received_wavelength / (z + 1) # Object wavelength in nm


    sample['band'] = band + 1

    sample['hist'] = np.zeros((flux.shape[0], 8), dtype='float32')
    sample['hist'][:,0] = mjd
    sample['hist'][:,1] = flux
    sample['hist'][:,2] = flux_err
    sample['hist'][:,3] = detected

    sample['hist'][:,6] = (source_wavelength/1000)
    sample['hist'][:,7] = (received_wavelength/1000)

    set_intervals(sample)
    
    flux_max = np.max(flux)
    flux_min = np.min(flux)
    flux_pow = math.log2(flux_max - flux_min)
    sample['hist'][:,1] /= math.pow(2, flux_pow)
    sample['hist'][:,2] /= math.pow(2, flux_pow)
    sample['meta'][9] = flux_pow / 10
    
    samples.append(sample)

    if len(samples) % 1000 == 0:
        print('Converting data {0}'.format(len(samples)), end='\r')

    if len(samples) >= limit:
        break
    
  

In [ ]:
target

In [ ]:
 np.where(classes == int(target))[0][0] # positional index of the classes array

In [ ]:
i=object_vectors.get(615)

In [ ]:
a[[0,1,3], :][:, [0,2]]  # Selects the columns you want as well

In [ ]:
meta=meta=np.array(i.get('meta'), dtype='float32')
meta

In [ ]:
meta[4]

In [ ]:
j[0][0]

In [ ]:
i=dict_persons.get(6266)

In [ ]:
dict_persons.values()

In [ ]:
i

In [ ]:
[*i]

In [ ]:
for key in i.keys():
  print(key)

In [ ]:
i.get('object_id')

In [ ]:
np.array(i.get('band')).shape

In [ ]:
j=i.get('hist')

In [ ]:
j

In [ ]:
for row, sublist in enumerate(j):
    for column, item in enumerate(sublist):
        if item:
            print((row, column))
            


In [ ]:
j[0][0] -- mjd

In [ ]:
j[0][1] # passband

In [ ]:
j[0][2] # flux

In [ ]:
j[0][3] # flux_err

In [ ]:
j[0][4] # Detected

In [ ]:
for row, sublist in enumerate(k):
    for column, item in enumerate(sublist):
        if item:
            print((row, column))
            

In [ ]:
k[0][1]

In [ ]:

print(train_x['id'].shape)
print(train_x['meta'].shape)
print(train_x['band'].shape)
print(train_x['hist'].shape)

OK, so for hist - each row, we have 8 arrays palanced out to 256 entries - pad stuff! and the arrays are 0 mjd 1 flux 2 flux_err 3 etc 4 etc 5 etc 6 7

so 

- train_x['hist'][0,:,0] is mjd
- train_x['hist'][0,:,1] is flux
- train_x['hist'][0,:,2] is flux_err
- train_x['hist'][0,:,3] is detected
- train_x['hist'][0,:,4] is mjd_deltas
- train_x['hist'][0,:,5] is mjd_reverse_deltas
- train_x['hist'][0,:,6] is source_wavelength
- train_x['hist'][0,:,7] is received_wavelength

Just remember that in get_keras_data, mjd, detected and received_wavelength values are removed and set to zero.

In [ ]:
train_x['id'][0]

In [ ]:
train_x['hist'][0,:,4]

In [ ]:
mjd=[1,2,4,8,16,32,64]

mjd1=np.ediff1d(mjd, to_begin = [0]) - mjd deltas between elements
mjd2=np.ediff1d(mjd, to_end = [0]) - reverse mjd from end

#hist[:,4] = np.ediff1d(hist[:,0], to_begin = [0])
#hist[:,5] = np.ediff1d(hist[:,0], to_end = [0])

In [ ]:
fullSetSQL="""
select * from training_set_all_padded
"""

In [ ]:
fullDF =sqlContext.sql(fullSetSQL)

In [ ]:
fullDF.printSchema()

In [ ]:
fullDF.count()

In [ ]:
X = {
        'id': np.array(fullDF.select('object_id').collect(), dtype='int32'),
        'meta': np.array(fullDF.select('meta').collect(), dtype='float32'),
        'band': np.array(fullDF.select('band').collect() , dtype='int32').reshape(7848,256)
#        'hist': pad_sequences([i['hist'] for i in itemslist], maxlen=sequence_len, dtype='float32'),
    }
#print('creating Y')
#Y = to_categorical([i['target'] for i in itemslist], num_classes=len(classes))

In [ ]:
X['id'].shape

In [ ]:
histTest=np.array(fullDF.select('hist.interval').collect(), dtype='float32').reshape(7848,256)

In [ ]:
histTest.shape

In [ ]:
fluxTest=np.array(fullDF.select('hist.flux').collect(), dtype='float32').reshape(7848,256)
flux_err_test=np.array(fullDF.select('hist.flux_err').collect(), dtype='float32').reshape(7848,256)

In [ ]:
flux_err_test.shape

In [ ]:
histArray=np.zeros((7848,256,8), dtype='float32') 
# this will work brilliantly as get_keras_data sets three columns to zeros anyway

In [ ]:
histArray[:,:,0]=histTest
histArray[:,:,1]=fluxTest
histArray[:,:,1]=flux_err_test

In [ ]:
histArray.shape

In [ ]:
targetArray=fluxTest=np.array(fullDF.select('target').collect(), dtype='int32')

In [ ]:
targetArray.shape

In [ ]:
classes

In [ ]:
len(classes)

In [ ]:
np.where(classes == 6)

In [ ]:
Y = to_categorical(3, num_classes=len(classes))

In [ ]:
Y

In [ ]:
fullDF.select('meta').collect()

# Experimental - testing mods for get data below

In [ ]:
extragalactic=None

In [ ]:
samples = []
# You have to perform an aggregation on the Spark dataframe and collect the results before you can iterate 
# This i not necessary with pandas
groups = train_mdf_data.groupby('object_id')


In [ ]:
for g in groups:
    id=g[0]
    
    sample = {}
    sample['id'] = int(id)
    
    meta = train_meta.loc[train_meta['object_id'] == id]
    
    # NEW We need to get the arrays for each pivoted dataframe
    #mjd = train_mjd_data.loc[train_mjd_data['object_id'] == id]
    #passband=train_passband_data.loc[train_passband_data['object_id'] == id]
    #flux=train_flux_data.loc[train_flux_data['object_id'] == id]
    #flux_err=train_flux_err_data.loc[train_flux_err_data['object_id'] == id]
    #detected=train_detected_data.loc[train_detected_data['object_id'] == id]
    
    if extragalactic == True and float(meta['hostgal_photoz']) == 0:
        continue

    if extragalactic == False and float(meta['hostgal_photoz']) > 0:
        continue    
    
    if 'target' in meta:
        sample['target'] = np.where(classes == int(meta['target']))[0][0]
    else:
        sample['target'] = len(classes) - 1   

    sample['meta'] = np.zeros(10, dtype = 'float32')

    sample['meta'][4] = meta['ddf']
    sample['meta'][5] = meta['hostgal_photoz']
    sample['meta'][6] = meta['hostgal_photoz_err']
    sample['meta'][7] = meta['mwebv']
    sample['meta'][8] = float(meta['hostgal_photoz']) > 0

    sample['specz'] = float(meta['hostgal_specz'])

    
    if use_specz:
        sample['meta'][5] = float(meta['hostgal_specz'])
        sample['meta'][6] = 0.0

    z = float(sample['meta'][5])
    
    # we need to drop the object_id from the pivot records. We can use any of the pivot dataframes,
    # because they all have the same shape, coming from a Hive table definition. We'll use the MJD
    # pivot dataframe to set up the indexes we want. How we do this - in stages
    # 1. Create a data frame for the pivot dataframes, on for each object_id
    # 2. Use dropna to remove NAN column values
    # 3. Cast that dataframe to a numpy array and get the shape
    # 4. Use the mjd array as a base, create an index list of the columns we want - ie we're dropping the object_id
    # 5. Use the index ro truncate the object_id column from the rest of the arrays
    # 6. finally, we need to reshape the arrays from [1;cols] to [cols,]   
    
    mjd = np.array(train_mjd_data.loc[train_mjd_data['object_id'] == id].dropna(axis='columns'), dtype='float32')
    r,c=mjd.shape

    idx_OUT_columns = [0]
    idx_IN_columns = [i for i in range(np.shape(mjd)[1]) if i not in idx_OUT_columns]

    mjd = mjdArray[:,idx_IN_columns].reshape(c-1,)
    band = np.array(train_passband_data.loc[train_passband_data['object_id'] == id].dropna(axis='columns') , dtype='int32')[:,idx_IN_columns].reshape(c-1,)
    flux = np.array(train_flux_data.loc[train_flux_data['object_id'] == id].dropna(axis='columns') , dtype='float32')[:,idx_IN_columns].reshape(c-1,)
    flux_err = np.array(train_flux_err_data.loc[train_flux_err_data['object_id'] == id].dropna(axis='columns') , dtype='float32')[:,idx_IN_columns].reshape(c-1,)
    detected = np.array(train_detected_data.loc[train_detected_data['object_id'] == id].dropna(axis='columns') , dtype='float32')[:,idx_IN_columns].reshape(c-1,)
    #mjd      = np.array(g[1]['mjd'],      dtype='float32')
    #band     = np.array(g[1]['passband'], dtype='int32')
    #flux     = np.array(g[1]['flux'],     dtype='float32')
    #flux_err = np.array(g[1]['flux_err'], dtype='float32')
    #detected = np.array(g[1]['detected'], dtype='float32')  

    
    mjd -= mjd[0]
    mjd /= 100 # Earth time shift in day*100
    mjd /= (z + 1) # Object time shift in day*100

    
    received_wavelength = passbands[band] # Earth wavelength in nm
    received_freq = 300000 / received_wavelength # Earth frequency in THz
    source_wavelength = received_wavelength / (z + 1) # Object wavelength in nm

    
    sample['band'] = band + 1

    sample['hist'] = np.zeros((flux.shape[0], 8), dtype='float32')
    sample['hist'][:,0] = mjd
    sample['hist'][:,1] = flux
    sample['hist'][:,2] = flux_err
    sample['hist'][:,3] = detected

    sample['hist'][:,6] = (source_wavelength/1000)
    sample['hist'][:,7] = (received_wavelength/1000)
    
    set_intervals(sample)


    flux_max = np.max(flux)
    flux_min = np.min(flux)
    flux_pow = math.log2(flux_max - flux_min)
    sample['hist'][:,1] /= math.pow(2, flux_pow)
    sample['hist'][:,2] /= math.pow(2, flux_pow)
    sample['meta'][9] = flux_pow / 10
    
    samples.append(sample)

    if len(samples) % 1000 == 0:
        print('Converting data {0}'.format(len(samples)), end='\r')

    if len(samples) >= limit:
        break


In [ ]:
sample

In [ ]:
id=713
print(id)
sample = {}
sample['id'] = int(id)

meta = train_meta.loc[train_meta['object_id'] == id]

# NEW We need to get the arrays for each pivoted dataframe
#mjd = train_mjd_data.loc[train_mjd_data['object_id'] == id]
#passband=train_passband_data.loc[train_passband_data['object_id'] == id]
#flux=train_flux_data.loc[train_flux_data['object_id'] == id]
#flux_err=train_flux_err_data.loc[train_flux_err_data['object_id'] == id]
#detected=train_detected_data.loc[train_detected_data['object_id'] == id]

if extragalactic == True and float(meta['hostgal_photoz']) == 0:
    print('Hi there')

if extragalactic == False and float(meta['hostgal_photoz']) > 0:
    print('Hi there again')  

if 'target' in meta:
    sample['target'] = np.where(classes == int(meta['target']))[0][0]
else:
    sample['target'] = len(classes) - 1   

sample['meta'] = np.zeros(10, dtype = 'float32')

sample['meta'][4] = meta['ddf']
sample['meta'][5] = meta['hostgal_photoz']
sample['meta'][6] = meta['hostgal_photoz_err']
sample['meta'][7] = meta['mwebv']
sample['meta'][8] = float(meta['hostgal_photoz']) > 0

sample['specz'] = float(meta['hostgal_specz'])


if use_specz:
    sample['meta'][5] = float(meta['hostgal_specz'])
    sample['meta'][6] = 0.0

z = float(sample['meta'][5])

# we need to drop the object_id from the pivot records. We can use any of the pivot dataframes,
# because they all have the same shape, coming from a Hive table definition. We'll use the MJD
# pivot dataframe to set up the indexes we want. How we do this - in stages
# 1. Create a data frame for the pivot dataframes, on for each object_id
# 2. Use dropna to remove NAN column values
# 3. Cast that dataframe to a numpy array and get the shape
# 4. Use the mjd array as a base, create an index list of the columns we want - ie we're dropping the object_id
# 5. Use the index ro truncate the object_id column from the rest of the arrays
# 6. finally, we need to reshape the arrays from [1;cols] to [cols,],
mjd = np.array(train_mjd_data.loc[train_mjd_data['object_id'] == id].dropna(axis='columns'), dtype='float32')
r,c=mjd.shape

idx_OUT_columns = [0]
idx_IN_columns = [i for i in range(np.shape(mjd)[1]) if i not in idx_OUT_columns]

mjd = mjdArray[:,idx_IN_columns].reshape(c-1,)
band = np.array(train_passband_data.loc[train_passband_data['object_id'] == id].dropna(axis='columns') , dtype='int32')[:,idx_IN_columns].reshape(c-1,)
flux = np.array(train_flux_data.loc[train_flux_data['object_id'] == id].dropna(axis='columns') , dtype='float32')[:,idx_IN_columns].reshape(c-1,)
flux_err = np.array(train_flux_err_data.loc[train_flux_err_data['object_id'] == id].dropna(axis='columns') , dtype='float32')[:,idx_IN_columns].reshape(c-1,)
detected = np.array(train_detected_data.loc[train_detected_data['object_id'] == id].dropna(axis='columns') , dtype='float32')[:,idx_IN_columns].reshape(c-1,)

#mjd      = np.array(g[1]['mjd'],      dtype='float32')
#band     = np.array(g[1]['passband'], dtype='int32')
#flux     = np.array(g[1]['flux'],     dtype='float32')
#flux_err = np.array(g[1]['flux_err'], dtype='float32')
#detected = np.array(g[1]['detected'], dtype='float32')  

# Now we need to reshape to columns
#mjd=mjd.reshape(352,)

mjd -= mjd[0]
mjd /= 100 # Earth time shift in day*100
mjd /= (z + 1) # Object time shift in day*100




In [ ]:

received_wavelength = passbands[band] # Earth wavelength in nm
received_freq = 300000 / received_wavelength # Earth frequency in THz
source_wavelength = received_wavelength / (z + 1) # Object wavelength in nm

In [ ]:
hiThere = train_mjd_data.loc[train_mjd_data['object_id'] == id]

In [ ]:
train_mjd_data.

In [ ]:
hiThere

In [ ]:
hiThere.dropna(axis='columns')

In [ ]:
mjd = np.array(train_mjd_data.loc[train_mjd_data['object_id'] == id].dropna(axis='columns'), dtype='float32')

In [ ]:
r,c=mjd.shape

In [ ]:
r

In [ ]:
c